<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Networks (GAN)
## 目的
GANによって画像の生成をして動作を理解する．

## モジュールのインポート

はじめに必要となるモジュールをインポートします．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算を行うことが可能です．
Falseとなっている場合は，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

In [ ]:
import os
import zipfile
import urllib.request
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## ネットワークの構築

GANは，Generator (生成器)とDiscriminator (識別器)と呼ばれる２つのネットワークによって構成される生成モデルです．
そのため，2つのネットワークを別々に構築します．

Generatorは，$N(0, 1)$や$U[-1, 1]$からサンプリングした潜在変数を入力して画像を生成するネットワークです．Generatorは，綺麗な画像を生成することでDiscriminatorを欺くことを目的としています．

一方で，Discriminatorは実画像 (訓練画像)またはGeneratorが生成した画像のどちらかを入力して，入力されたデータを正確に判別するネットワークです．Discriminatorは，入力画像の中からGeneratorの生成した画像を見破ることを目的としています．

GANの最終的な目的は，実画像$x$の確率分布$p_{data}(x)$と実画像$x$をGenerator上の分布で見た時の確率分布$p_{g}(x)$が一致することです．つまり，$p_{data}(x)=p_{g}(x)$が成立した時にDiscriminatorが完全にRealなのかFakeなのかわからなくなっていると言えます．

GANのネットワークを簡易的に表現したものを，以下に示します．Generator及びDiscriminatorの構造は非常にシンプルで，全結合とReLUによって構築します．

<img src="https://dl.dropboxusercontent.com/s/deek34es6dqu4lb/gan.png" width=50%>

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 784))
        
    def forward(self, z):
        return self.layer(z)
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1))
    
    def forward(self, x):
        return self.layer(x)

## データセットと最適化関数
データセットにはMNISTを使用します．
最適化関数はAdam optimizer使用し，学習率$2\times 10^4$，betaの値を$0.5, 0.999$として学習します．

In [ ]:
# MNIST datasetの設定
transform_train = transforms.Compose([transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform_train, download=True)
training_data = DataLoader(mnist_data, batch_size=100, shuffle=True)

latent_dim = 100
G = Generator(latent_dim=latent_dim)
D = Discriminator()
if use_cuda:
    G = G.cuda()
    D = D.cuda()
opt_g = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_d = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

## GANの学習

次に，GANの学習をします．

GANの最適化式は以下に示す通りです．

$$
\min_{G}\max_{D}V(D, G) = \mathbb{E}_{x\sim P_{data}(x)}\left[\log\left(D(x)\right)\right] + \mathbb{E}_{z\sim P(z)}\left[\log\left(1 - D(\hat{x})\right)\right]
$$

ここで，$x$は実画像，$\hat{x}$がGeneratorの生成した画像G(z)に対応します．GANを学習する際は，binary cross entopyを用いて，実画像は1に，生成画像は0に近似するように学習をします．
Discriminatorは，実画像は1生成画像は0と識別するとように学習をしますが，Generatorは生成した画像を実画像であるとDiscriminatorに誤識別をさせたいので，1と識別されるように学習をします．
これによりGANにおける敵対学習を行うことができます．

`n_critic`は，Discriminatorを1 iterationあたり何回更新するかを制御する数となっています．
すなわち，Discriminatorを複数回更新した後にGeneratorを1回更新します．
この理由は，モード崩壊を防止するためです．モード崩壊とは，GANの学習における一般的な問題で，Generatorがある一定の画像しか生成できなることや全く画像が生成できなくなる問題を指します．

Discriminatorは，精度が高すぎても低すぎても学習が失敗してしまうため，適切なn_criticを指定する必要があります．n_criticが多いと簡単にモード崩壊します．一般的には，2回程度がよく用いられます．

In [ ]:
n_epoch = 10
n_critic = 2
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, n_epoch+1):
    Tensor = torch.cuda.FloatTensor
    for idx, (real_x, _) in enumerate(training_data):
        real_x = real_x.cuda()
        batch = real_x.size(0)
        flag_real = Tensor(batch, 1).fill_(1.0)
        flag_fake = Tensor(batch, 1).fill_(0.0)
        
        for _ in range(n_critic):
            D.zero_grad()

            z = torch.randn(batch, latent_dim)
            if use_cuda:
                z = z.cuda()
            
            fake_x = G(z)
            out_real = D(real_x.view(batch, -1))
            out_fake = D(fake_x.detach().view(batch, -1))
            loss_real = criterion(out_real, flag_real)
            loss_fake = criterion(out_fake, flag_fake)
            dis_loss = loss_real + loss_fake
            dis_loss.backward()
            opt_d.step()
            
        G.zero_grad()

        z = torch.randn(batch, latent_dim)
        if use_cuda:
            z = z.cuda()

        fake_x = G(z)
        out_gen = D(fake_x)
        gen_loss = criterion(out_gen, flag_real)
        gen_loss.backward()
        opt_g.step()
        
        if idx % 100 == 0:
            print('Training epoch: {} [{}/{} ({:.0f}%)] | D loss: {:.6f} | G loss: {:.6f} |'\
                  .format(epoch, idx * len(real_x), len(training_data.dataset),
                  100. * idx / len(training_data), dis_loss.item(), gen_loss.item()))

## 学習したGeneratorによる画像生成

学習したGeneratorを用いて画像を生成します．

ここでは，正規分布に従い乱数`z`を生成し，それをGeneratorへと入力することで，画像生成をおこない，その結果を表示します．

適切な学習が行われている場合は，0~9の数字の画像が生成されます．

In [ ]:
num_generate = 100
ch = 100
z = torch.randn(num_generate, ch)
if use_cuda:
    z = z.cuda()
test_img = G(z)

if use_cuda:
    test_img = test_img.cpu()
test_img_array = (test_img * 256.).clamp(min=0., max=255.).view(num_generate, 28, 28).data.numpy()

fig = plt.figure(figsize=(10, 10))
for i, im in enumerate(test_img_array):
    ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

# 課題

1. 潜在変数の次元数を100次元から減らした場合どうなるでしょうか．また，増やした場合はどのようになるでしょうか．

# 参考文献
[1] Ian Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville and Yoshua Bengio, Generative adversarial nets, NIPS, 2014.